<a href="https://colab.research.google.com/github/simgyojin/2020-Bigdata-Finance-Challenge/blob/main/%EC%99%B8%EB%B6%80%EB%8D%B0%EC%9D%B4%ED%84%B0/%EC%BD%94%EB%93%9C/%ED%96%89%EC%A0%95%EA%B5%AC%EC%97%AD_%EB%A9%B4%EC%A0%81_%EA%B0%80%EA%B3%B5_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#행정구역 면적 가공 코드
###행정구역_면적.csv를 행정코드별_면적.csv로 가공

In [ ]:
import pandas as pd
from google.colab import files
import re
get_num = re.compile('\d+')
files.upload()

In [ ]:
#필요한 데이터 파일
district_df = pd.read_csv('행정구역(시군구)면적.csv', encoding='CP949')
discode_df=pd.read_csv('행정동_20200814.csv')#, encoding='utf-8')
popul_df=pd.read_csv('201904_05_인구현황.csv')

#컬럼 명 [시군구명]으로 통일
columns={'소재지(시군구)별(1)':'시도명','소재지(시군구)별(2)':'시군구명'}
district_df.rename(columns = columns, inplace=True)

#행정구역 행정코드로 전처리
for i in  range(len(popul_df)):
  popul_df['행정구역'][i] = int(get_num.findall(popul_df['행정구역'][i])[0])

#필요없는 column 제거
popul_df.drop(['2019년04월_세대수',	'2019년04월_세대당 인구',	'2019년05월_총인구수',	
               '2019년05월_세대수',	'2019년05월_세대당 인구'], axis='columns', inplace=True)

In [ ]:
#합집합
merge_outer = pd.merge(discode_df,district_df, on='시군구명')

#필요없는 column제거
merge_outer.drop(['2019.1','2019.2','시도명_y','생성일자','말소일자'], axis='columns', inplace=True)

#null값 제거  
#시군구 수준이 지워져서 null값제거X
#merge_outer=merge_outer.dropna(axis=0)

#column 이름 변경
columns={'2019':'면적(km^2)','시도명_x':'시도명'}
merge_outer.rename(columns = columns, inplace=True)

cc = {'행정구역':'행정동코드','2019년04월_총인구수':'인구수'}
popul_df.rename(columns = cc, inplace=True)

In [ ]:
#합집합
last_merge = pd.merge(merge_outer,popul_df, on='행정동코드')

#123,232과 같은 형식 int형식으로 변경
for i in range(len(last_merge)):
  last_merge['인구수'][i] = int(last_merge['인구수'][i].replace(',',''))
  last_merge['면적(km^2)'][i] = int(last_merge['면적(km^2)'][i].replace(',',''))*0.001

#인구밀도 column추가
last_merge['인구밀도']=last_merge['인구수']/last_merge['면적(km^2)']

In [ ]:
#csv파일로 저장
last_merge.to_csv('인구밀도_가공_2019기준.csv',index=False, encoding='utf-8')

#file다운로드
files.download('인구밀도_가공_2019기준.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>